In [ ]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent.parent.absolute()))

In [ ]:
from multiversx_sdk_core import Address, Transaction, TransactionPayload
from multiversx_sdk_core.transaction_builders import DefaultTransactionBuildersConfiguration
from multiversx_sdk_core import TokenPayment
from multiversx_sdk_network_providers import ProxyNetworkProvider
from tools.notebooks import env

config = DefaultTransactionBuildersConfiguration(chain_id="1")
proxy_mainnet = ProxyNetworkProvider("https://gateway.multiversx.com")
proxy_shadowfork = ProxyNetworkProvider(env.PROXY_URL)

In [ ]:
from multiversx_sdk_core import AccountNonceHolder
from multiversx_sdk_wallet import UserSigner
from pathlib import Path

user_signer = UserSigner.from_pem_file(Path(env.USER1))

clone & forward transaction

In [ ]:
hash = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"

original_tx_data = proxy_mainnet.get_transaction(hash)
print(original_tx_data.data)
forwarded_tx  = Transaction(
    sender=original_tx_data.sender,
    receiver=original_tx_data.receiver,
    value=TokenPayment.egld_from_integer(original_tx_data.value),
    data=TransactionPayload.from_str(original_tx_data.data),
    chain_id=config.chain_id,
    gas_limit=original_tx_data.gas_limit,
    gas_price=config.min_gas_price
)

print(forwarded_tx.to_dictionary())

In [ ]:
user_nonce_holder = AccountNonceHolder(proxy_shadowfork.get_account(forwarded_tx.sender).nonce)
forwarded_tx.nonce = user_nonce_holder.get_nonce_then_increment()
forwarded_tx.signature = user_signer.sign(forwarded_tx)
proxy_shadowfork.send_transaction(forwarded_tx)